In [1]:
import praw

In [2]:
import pickle

In [9]:
import time
import datetime
from calendar import monthrange


In [4]:
import numpy as np

In [5]:
reddit = praw.Reddit(client_id='qJwY2ZzRahm-GQ',
                     client_secret='lkZjzt2g6MutrTm3D7c7hNl5R6w',
                     user_agent='linux:reddit-analysis:v1.0.0 (by /u/DigitalArcheology)',
                     username='DigitalArcheology',
                     password='GOvols!2017')

In [6]:
sub = reddit.subreddit('worldnews')

In [7]:
startYear, startMonth = 2017, 1
stopYear, stopMonth = 2017, 1

In [11]:
timestamp_Start = time.mktime(datetime.datetime(startYear, startMonth, 1, 0, 0, 0).timetuple())
timestamp_Stop = time.mktime(datetime.datetime(stopYear, stopMonth, monthrange(stopYear, stopMonth)[1], 23, 59, 59, 99).timetuple())
print('Start:', timestamp_Start)
print('Stop:', timestamp_Stop)

Start: 1483228800.0
Stop: 1485907199.0


In [20]:
def load_dates(path):
    dates_D = {}
    with open(path, 'r', newline='\n') as f:
        for line in f:
            reddit_id, field = line.strip().split('\t', maxsplit=1)
            dates_D[reddit_id] = field
    return dates_D

def load_data(path, dates_D):
    reddit_ids = []
    data = []
    with open(path, 'r', newline='\n') as f:
        for line in f:
            reddit_id, comments = line.split('\t', maxsplit=1)
            if reddit_id in dates_D.keys():
                data.append(comments)
                reddit_ids.append(reddit_id)
    return reddit_ids, data

In [26]:
dates_D = load_dates('../data/worldnews_2017_1-2017_1_submissiondates.txt')
reddit_ids, data = load_data('../data/worldnews_2017_1-2017_1_flatcomments.txt', dates_D)

In [27]:
len(reddit_ids)

623

In [29]:
sub.controversial?

In [31]:
for i, id_ in enumerate(sub.controversial('month')):
    if i < 10:
        print(i, id_)

0 7bd444
1 7duovf
2 78m1l4
3 7c5h47
4 7ax0fp
5 7aj2bc
6 77nysb
7 7bqxtj
8 7a2ob2
9 7bkag8


In [27]:
print(id_)

789tgq


In [ ]:
for submission in sub.submissions(start=timestamp_Start, end=timestamp_Stop):